# Training a model to predict future personal consumption

## Scrape the data

Gather data from https://fred.stlouisfed.org/ on Personal Consumption Expenditures (PCE) each month from January 2013 to May 2021.

### Import necessary libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

### Create a ChromeDriver Instance

In [ ]:
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options=options)

### Get data

In [ ]:
data = {'Month': [], 'PCE': []}
date = datetime.datetime(2013, 1, 1)
for i in range(101):
    year = date.year
    month = str(date.month).zfill(2)
    url = "https://fred.stlouisfed.org/release/tables?rid=54&eid=3220&od={}-{}-01#".format(year,month)
    driver.get(url)
    pce = driver.find_element_by_xpath('//*[@id="release-elements-tree"]/tbody/tr[1]/td[4]')
    print(pce.text)
    data['Month'].append(month + '-' + str(year))
    data['PCE'].append(pce.text)
    date = date + relativedelta(months=1)
driver.quit()

### Create a Pandas DataFrame

In [ ]:
df = pd.DataFrame(data)
df.to_pickle("df.pkl") #saves the dataframe locally
print(df)

## Preprocessing the Data

### Import the necessary libraries

In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

### Inspect and Clean the Dataset

In [ ]:
df = pd.read_pickle("df.pkl")
df.shape

In [ ]:
month_col = pd.to_datetime(df.pop('Month'), format='%m-%Y')
month_col

In [ ]:
df["PCE"] = df["PCE"].str.replace(",","").astype(float)

In [ ]:
df.head()

In [ ]:
plot_cols = ['PCE']
plot_features = df[plot_cols]
plot_features
plot_features.index = month_col
_ = plot_features.plot(subplots=True)

In [ ]:
df.describe().transpose()

In [ ]:
timestamp_s = month_col.map(datetime.datetime.timestamp)
timestamp_s

### Split the data

In [ ]:
n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

### Normalize the data

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [ ]:
df_std = (df - train_mean) / train_std
df_std = df_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(df.keys(), rotation=90)

### Data Windowing

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
        labels = tf.stack(
            [labels[:, :, self.column_indices[name]] for name in self.label_columns], axis = -1)
    
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

WindowGenerator.split_window = split_window

In [ ]:
def plot(self, model=None, plot_col='PCE', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Month')

WindowGenerator.plot = plot

### Creating the Dataset

In [ ]:
def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(data=data, targets=None, sequence_length=self.total_window_size, sequence_stride=1, shuffle=True, batch_size=32,)
    ds = ds.map(self.split_window)
    return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
    return self.make_dataset(self.train_df)

@property
def val(self):
    return self.make_dataset(self.val_df)

@property
def test(self):
    return self.make_dataset(self.test_df)

@property
def example(self):
    # Example batch of inputs and labels
    result = getattr(self, '_example', None)
    if result is None:
        result = next(iter(self.train))
        self._example = result
    return result 

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example   

### Building the model

In [ ]:
OUT_STEPS = 3
multi_window = WindowGenerator(input_width=6, label_width=OUT_STEPS, shift=OUT_STEPS)

multi_window.plot()
multi_window

In [ ]:
CONV_WIDTH = 3
conv_model = tf.keras.Sequential([
    # tf.keras.layers.LSTM(50, return_sequences=False),
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    # tf.keras.layers.Conv2D(512, activation='relu', kernel_size=(CONV_WIDTH)),
    # Shape => [batch, 1, out_steps]
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(OUT_STEPS, kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, 1])
])

In [ ]:
MAX_EPOCHS = 20
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
conv_model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(), metrics=[tf.metrics.MeanAbsoluteError()])
conv_model.fit(multi_window.train, epochs=MAX_EPOCHS, validation_data=multi_window.val, callbacks=[early_stopping])

In [ ]:
val_performance = conv_model.evaluate(multi_window.val)
val_performance

In [ ]:
test_performance = conv_model.evaluate(multi_window.test, verbose=0)
test_performance

In [ ]:
multi_window.val

In [ ]:
multi_window.plot(conv_model)

### Saving the model

In [ ]:
conv_model.save('./model.h5')